# This example demonstrate how to train YOLO detector model

In [1]:
import sys
import os
import json
import yaml
import shutil
import matplotlib.pyplot as plt

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../../../../')
sys.path.append(NOMEROFF_NET_DIR)

from nomeroff_net.tools.yolo_tools import convert_dataset_to_yolo_format

In [2]:
plt.rcParams["figure.figsize"] = (20, 10)

In [3]:
%matplotlib inline 

# Download dataset and yolo repo

In [4]:
# auto download latest dataset
from nomeroff_net.tools import modelhub

# get yolov5
info = modelhub.download_repo_for_model("yolov5")
repo_path = info["repo_path"]

# auto download latest dataset
info = modelhub.download_dataset_for_model("yolov5")
PATH_TO_DATASET = info["dataset_path"]

# local path dataset
# PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, "./data/dataset/Detector/autoria_numberplate_dataset_example")

# Convert dataset to yolo format

In [5]:
CLASSES = ['numberplate', 'brand_numberplate']
STATES  = ["val","train"]
#STATES  = ["val"]

PATH_TO_RES_ANN = os.path.join(os.path.dirname(PATH_TO_DATASET), "./npdata/labels/{}")
PATH_TO_RES_IMAGES  = os.path.join(os.path.dirname(PATH_TO_DATASET), "./npdata/images/{}")

PATH_TO_JSON = os.path.join(PATH_TO_DATASET, "{}/via_region_data.json")
PATH_TO_IMAGES = os.path.join(PATH_TO_DATASET, "{}/")

In [6]:
EXIST_STRATEGY = "exist_ok" # "delete", "exist_ok"

for state in STATES:
    path_to_res_ann    = PATH_TO_RES_ANN.format(state)
    path_to_res_images = PATH_TO_RES_IMAGES.format(state)
    
    if os.path.exists(path_to_res_ann) and os.path.exists(path_to_res_images) and EXIST_STRATEGY == "exist_ok":
        print("[INFO]", state, "data exists")
        continue
    if EXIST_STRATEGY == "delete":
        shutil.rmtree(path_to_res_ann)
        shutil.rmtree(path_to_res_images)
    
    print("[INFO]", state, "data creating...")
    os.makedirs(path_to_res_ann, exist_ok=True)
    os.makedirs(path_to_res_images, exist_ok=True)
    
    path_to_json    = PATH_TO_JSON.format(state)
    path_to_images  = PATH_TO_IMAGES.format(state)
    
    convert_dataset_to_yolo_format(
        path_to_res_ann, 
        path_to_res_images,
        path_to_images, 
        path_to_json, 
        debug=False,
        classes=CLASSES,
        is_generate_image_rotation_variants=False
    )
    

  0%|          | 2/563 [00:00<00:37, 14.79it/s]

[INFO] val data creating...


  0%|          | 2/8412 [00:00<09:44, 14.40it/s]

[INFO] find labels ['brand_numberplate', 'numberplate']
[INFO] use labels ['numberplate', 'brand_numberplate']
[INFO] count attributes 592
[INFO] skip attributes 0
[INFO] train data creating...


100%|██████████| 8412/8412 [11:06<00:00, 12.63it/s]

[INFO] find labels ['brand_numberplate', 'numberplate']
[INFO] use labels ['numberplate', 'brand_numberplate']
[INFO] count attributes 8993
[INFO] skip attributes 0


# Create yaml config

In [7]:
yaml_path = os.path.join(os.path.dirname(PATH_TO_DATASET), "./npdata/numberplate_config.yaml")
yaml_dataset_config = {
    # Train/val/test sets
    "path": os.path.dirname(PATH_TO_DATASET), 
    "train": "./npdata/images/train",
    "val": "./npdata/images/val",
    #"test": "" # test images (optional)
    
    # Classes
    "nc": len(CLASSES),  # number of classes
    "names": CLASSES  # class names
}
print("[INFO] config:", yaml_dataset_config)
print("[INFO] config saved into", yaml_path)
with open(yaml_path, 'w') as yaml_file:
    yaml.dump(yaml_dataset_config, 
              yaml_file, 
              default_flow_style=False)

[INFO] config: {'path': '/mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/Detector/yolov5', 'train': './npdata/images/train', 'val': './npdata/images/val', 'nc': 2, 'names': ['numberplate', 'brand_numberplate']}
[INFO] config saved into /mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/Detector/yolov5/./npdata/numberplate_config.yaml


# CLI yolo training

In [8]:
!cd {repo_path}; \
 wandb offline; \
 python3 train.py \
    --img 640 \
    --batch 1 \
    --epochs 1 \
    --data {yaml_path} \
    --weights yolov5s.pt

/bin/bash: wandb: command not found
train: weights=yolov5s.pt, cfg=, data=/mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/Detector/yolov5/./npdata/numberplate_config.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=1, batch_size=1, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), 11.48 KiB | 1.04 MiB/s, done.
From https://github.com/ultr

In [9]:
repo_path

'/mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./repos/Detector/yolov5'

## Run wandb localy

```
docker run --rm -it -v wandb:{repo_path}/wandb -p 8080:8080 --ip 0.0.0.0 --name wandb-local wandb/local
```